<a href="https://colab.research.google.com/github/AlchemicRonin/CS-498-Introduction-to-Deep-Learning/blob/TY/CS_498DL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount= True)

import os
os.chdir("/content/drive/My Drive/MP1")

Mounted at /content/drive/


In [2]:
!ls


 cifar10		        kaggle		       mushroom
'CS 498DL Assignment-1.ipynb'   kaggle_submission.py   __pycache__
 data_process.py	        models


In [3]:
import random
import numpy as np
from data_process import get_CIFAR10_data, get_MUSHROOM_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [4]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [5]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [6]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

# Loading Mushroom

In the following cells we determine the splitting of the mushroom dataset.
<br /> TRAINING + VALIDATION = 0.8, TESTING = 0.2

In [7]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2

In [8]:
data = get_MUSHROOM_data(VALIDATION)
X_train_MR, y_train_MR = data['X_train'], data['y_train']
X_val_MR, y_val_MR = data['X_val'], data['y_val']
X_test_MR, y_test_MR = data['X_test'], data['y_test']
n_class_MR = len(np.unique(y_test_MR))

print("Number of train samples: ", X_train_MR.shape[0])
print("Number of val samples: ", X_val_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[1])

Number of train samples:  4874
Number of val samples:  1625
Number of test samples:  1625
Number of test samples:  22


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [9]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/Perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on CIFAR

In [32]:
lr = 0.5
n_epochs = 100
w = X_train_CIFAR.shape[1]
percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs,w)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

array([[  -508.84486026,  -3896.44467843, -10216.90609818, ...,
          -440.53407449,    888.01873623, -20570.86823772],
       [ 16372.33407233,   6234.07514826,   -975.00518633, ...,
         -3691.23925029,  -2693.39806048,   6327.21947773],
       [  9795.7416032 ,  12571.76726425,   9679.2546448 , ...,
         11717.64612615, -21333.09421315,  -7093.88858419],
       ...,
       [ -1777.42256722,  -8724.15452433,  -7744.17834777, ...,
         -5322.31107906, -20951.46991053,  -5683.35709851],
       [ -2014.94804031,   2753.93050201,  -3254.10158572, ...,
          7142.06017381,  17179.33627238,  16998.95304059],
       [-12102.98021937,   2252.2017634 ,  11144.32377625, ...,
         -7372.94564998,  19737.26921567,  20667.81744426]])

In [33]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 33.495000


### Validate Perceptron on CIFAR

In [34]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 27.120000


### Test Perceptron on CIFAR

In [35]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 26.890000


### Perceptron_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [36]:
output_submission_csv('kaggle/perceptron_submission_CIFAR.csv', percept_CIFAR.predict(X_test_CIFAR))

## Train Perceptron on Mushroom

In [37]:
lr = 0.5
n_epochs = 50
w = X_train_MR.shape[1]
percept_MR = Perceptron(n_class_MR, lr, n_epochs,w)
percept_MR.train(X_train_MR, y_train_MR)

array([[  -1.20626972,  -17.59409404,    4.15601908,   -5.08553835,
          17.99398532,    3.72723814,  225.26632612, -288.92277624,
           3.20530407,    2.97873701,   75.36151728,  187.79478629,
          13.70893377,    7.57925594,    4.96729386,    0.29223975,
        -221.61593856,  -19.11489224,  -27.65302276,   21.29125442,
          18.86168809,   -4.16745755],
       [   2.6673705 ,   19.14385271,   -3.26381248,    5.81060936,
         -16.52216435,   -2.23809629, -224.23724863,  289.76526685,
          -2.37716115,   -2.12013807,  -74.18182349, -186.60573732,
         -13.02108137,   -7.17790357,   -3.69718867,    0.99684189,
         222.83127811,   19.58687305,   28.91057094,  -20.095151  ,
         -18.23952981,    4.85931444]])

In [38]:
pred_percept = percept_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_MR)))

The training accuracy is given by: 85.597046


### Validate Perceptron on Mushroom

In [39]:
pred_percept = percept_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_MR)))

The validation accuracy is given by: 83.200000


### Test Perceptron on Mushroom

In [40]:
pred_percept = percept_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_MR)))

The testing accuracy is given by: 85.353846


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with :
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/SVM.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on CIFAR

In [48]:
lr = 0.5
n_epochs = 50
reg_const = 0.05
w = X_train_CIFAR.shape[1]
svm_CIFAR = SVM(n_class_CIFAR, lr, n_epochs, reg_const,w)
svm_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

array([[131398.12487945, 120768.40682363, 111868.95107413, ...,
        323308.55808934, 322245.11466985, 318118.40775657],
       [140522.93591391, 113451.89318909, 111209.75547633, ...,
        322633.48764581, 311740.91920807, 321941.72403363],
       [113643.51435053, 142166.57246705, 142238.26856869, ...,
        322024.87608408, 299568.44263922, 304218.03597846],
       ...,
       [129331.35172074, 124170.43880508, 126923.42596755, ...,
        301392.08887585, 293866.30908386, 325006.7033962 ],
       [143203.30162247, 130699.60492952, 130332.58819315, ...,
        322414.8744645 , 330197.12238061, 339707.35748423],
       [123995.6325977 , 109798.26667051, 119531.99263895, ...,
        319629.51747004, 330860.25276133, 331096.93807661]])

In [49]:
pred_svm = svm_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_CIFAR)))

The training accuracy is given by: 30.607500


### Validate SVM on CIFAR

In [47]:
pred_svm = svm_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_CIFAR)))

The validation accuracy is given by: 27.520000


### Test SVM on CIFAR

In [44]:
pred_svm = svm_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_CIFAR)))

The testing accuracy is given by: 29.140000


### SVM_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/svm_submission_CIFAR.csv', svm_CIFAR.predict(X_test_CIFAR))

## Train SVM on Mushroom

In [52]:
lr = 0.5
n_epochs = 50
reg_const = 0.05
w = X_train_MR.shape[1]
svm_MR = SVM(n_class_MR, lr, n_epochs, reg_const,w)
svm_MR.train(X_train_MR, y_train_MR)

array([[1.14313771e+04, 6.63727967e+03, 1.82468863e+04, 2.18154553e+03,
        1.34279142e+04, 3.52088305e+03, 6.16500760e+02, 1.32097410e+03,
        2.15697645e+04, 1.25289577e+03, 5.02598573e+03, 6.51367452e+03,
        6.04386518e+03, 2.23561323e+04, 2.22494169e+04, 2.59427297e-01,
        7.03795760e+03, 3.90857468e+03, 1.13672715e+04, 1.10023359e+04,
        1.28922007e+04, 6.74360254e+03],
       [1.14297105e+04, 6.65112738e+03, 1.82434127e+04, 2.16216652e+03,
        1.34141975e+04, 3.52742391e+03, 4.60251076e+02, 1.52780844e+03,
        2.15661812e+04, 1.25552996e+03, 4.98388834e+03, 6.38432052e+03,
        6.03227675e+03, 2.23486618e+04, 2.22429975e+04, 2.19808860e-02,
        7.18666584e+03, 3.93041830e+03, 1.13863128e+04, 1.09888844e+04,
        1.28847449e+04, 6.74620532e+03]])

In [53]:
pred_svm = svm_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_MR)))

The training accuracy is given by: 82.970866


### Validate SVM on Mushroom

In [54]:
pred_svm = svm_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_MR)))

The validation accuracy is given by: 80.615385


## Test SVM on Mushroom

In [55]:
pred_svm = svm_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_MR)))

The testing accuracy is given by: 83.630769


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with :
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/Softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on CIFAR

In [56]:
lr = 0.5
n_epochs = 50
reg_const = 0.05
w = X_train_CIFAR.shape[1]
softmax_CIFAR = Softmax(n_class_CIFAR, lr, n_epochs, reg_const, w)
softmax_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

array([[ -766.85686492,   179.58578358, -5283.22671931, ...,
         -149.4954358 ,  1603.54858163, -4735.31338576],
       [ 4060.5790698 , -1742.16264493,    48.85860586, ...,
         1974.00878613,   480.7371653 ,  1985.97209381],
       [ 2385.87082138,  5669.25211107,  3649.76585034, ...,
         1248.89856867, -5438.22670579, -2313.54837636],
       ...,
       [  460.37179606, -1608.24700804,  -779.21013921, ...,
        -2893.53946068, -4058.13311245, -3144.99085686],
       [ 1278.33408336, -1229.80309231, -2704.08044242, ...,
          623.00507562,  5158.37337001,  4214.24360531],
       [-2579.53039377, -2741.28875924,  3559.45838359, ...,
         -532.45470078,  3628.54762742,  4762.79750282]])

In [57]:
pred_softmax = softmax_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_CIFAR)))

The training accuracy is given by: 31.277500


### Validate Softmax on CIFAR

In [59]:
pred_softmax = softmax_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_CIFAR)))

The validation accuracy is given by: 26.150000


### Testing Softmax on CIFAR

In [60]:
pred_softmax = softmax_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_CIFAR)))

The testing accuracy is given by: 26.290000


### Softmax_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [62]:
output_submission_csv('kaggle/softmax_submission_CIFAR.csv', softmax_CIFAR.predict(X_test_CIFAR))

## Train Softmax on Mushroom

In [63]:
lr = 0.5
n_epochs = 50
reg_const = 0.05
w = X_train_MR.shape[1]
softmax_MR = Softmax(n_class_MR, lr, n_epochs, reg_const, w)
softmax_MR.train(X_train_MR, y_train_MR)

array([[ 2.69519648e-01, -1.68305597e+01,  2.65903815e+00,
         1.49062010e+01,  9.70745056e+00, -9.71637932e+00,
         1.36171225e+02, -2.01623794e+02,  7.07990443e+00,
         1.86955417e+01,  4.84486382e+01,  1.09746906e+02,
         1.71988638e+01,  6.96649570e+00,  5.00397356e-01,
         2.52323158e-01, -1.53771942e+02, -6.88783438e+00,
        -1.16814816e+01,  2.18533278e+01,  6.72868906e+00,
        -4.36959926e+00],
       [ 1.20041415e-01,  1.70554781e+01, -2.34411606e+00,
        -1.46394733e+01, -9.54038815e+00,  9.89157521e+00,
        -1.35710034e+02,  2.01876927e+02, -6.72066636e+00,
        -1.84202586e+01, -4.79577731e+01, -1.09321324e+02,
        -1.71440420e+01, -6.79674345e+00, -1.99032003e-01,
         3.14602875e-03,  1.54021045e+02,  7.19011701e+00,
         1.19627503e+01, -2.16683203e+01, -6.24708162e+00,
         4.73299755e+00]])

In [64]:
pred_softmax = softmax_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_MR)))

The training accuracy is given by: 89.741485


### Validate Softmax on Mushroom

In [66]:
pred_softmax = softmax_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_MR)))

The validation accuracy is given by: 88.800000


### Testing Softmax on Mushroom

In [67]:
pred_softmax = softmax_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_MR)))

The testing accuracy is given by: 89.292308


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.



You will implement the Logistic Classifier in the **models/Logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [68]:
# Change labels to {-1,1}
for i in range(len(y_train_MR)):
  if (y_train_MR[i] == 0):
    y_train_MR[i] = -1
for i in range(len(y_val_MR)):
  if (y_val_MR[i] == 0):
    y_val_MR[i] = -1
for i in range(len(y_test_MR)):
  if (y_test_MR[i] == 0):
    y_test_MR[i] = -1

### Training Logistic Classifer

In [69]:
learning_rate = 0.5
n_epochs = 200

lr = Logistic(learning_rate, n_epochs)
lr.train(X_train_MR, y_train_MR)

array([[ 0.29182053],
       [ 1.52013397],
       [ 0.20205113],
       [-1.97476235],
       [-1.12383252],
       [ 1.24380113],
       [-1.47105249],
       [ 4.94161058],
       [-0.94301377],
       [-0.76020523],
       [-2.7752963 ],
       [-2.24957809],
       [-1.58600683],
       [-0.04801935],
       [ 0.17101631],
       [ 0.52001607],
       [ 1.56122728],
       [-0.47743611],
       [-1.5476569 ],
       [-0.40405846],
       [ 4.37471296],
       [ 1.46126741]])

In [70]:
pred_lr = lr.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_MR)))

The training accuracy is given by: 86.274108


### Validate Logistic Classifer

In [71]:
pred_lr = lr.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_MR)))

The validation accuracy is given by: 85.846154


### Test Logistic Classifier

In [72]:
pred_lr = lr.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_MR)))

The testing accuracy is given by: 84.246154
